In [1]:
#Import necessary packages
import pandas as pd
import numpy as np
import re
import gc
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
train = pd.read_csv('train.csv')

In [3]:
#Clean the text of punctuation, line breaks, capitalization, etc
def clean_text(contents):
    contents = re.sub("'", "", contents)
    cleaned_contents = re.sub('[^A-Za-z]+', ' ', contents)
    return re.sub('/\r?\n|\r/g', ' ', cleaned_contents).lower()

train.discourse_text = train.discourse_text.transform(clean_text)

In [4]:
#Find the count of each word and create a document-term matrix
cv = CountVectorizer(stop_words = 'english')
text_cv = cv.fit_transform(train.discourse_text)
dtm = pd.DataFrame(text_cv.toarray(), columns = cv.get_feature_names())

In [5]:
#Drop words that are  present in less than 1% of the text samples
word_counts = dtm.sum().where(dtm.sum() > (0.01*len(dtm.index)))
word_counts.dropna(inplace = True)
dtm = dtm.loc[:, word_counts.index.array]

In [6]:
#Index samples by discourse ID to ensure compatability with original training dataset
dtm['discourse_id'] = train['discourse_id']
dtm.to_csv('dtm.csv', header = True, index = False)

In [7]:
gc.collect()

75